In [3]:
import numpy as np
import pandas as pd
import os

In [4]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, Activation

In [5]:
num_classes = 81
img_rows, img_cols = 32, 32
batch_size = 16

In [6]:
train_dir = "./train/"
validation_dir = "./validation/"

In [7]:
# Data Augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=30,
                                  width_shift_range=0.3,
                                  height_shift_range=0.3,
                                  horizontal_flip=True,
                                  fill_mode="nearest")
validation_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(img_rows, img_cols),
                                                   batch_size=batch_size,
                                                   class_mode="categorical",
                                                   shuffle=True)
validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                   target_size=(img_rows, img_cols),
                                                   batch_size=batch_size,
                                                   class_mode="categorical",
                                                   shuffle=False)

Found 41322 images belonging to 81 classes.
Found 13877 images belonging to 81 classes.


In [9]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding="same", input_shape=(img_rows, img_cols, 3)))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation("softmax"))

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)       

In [12]:
from keras.optimizers import RMSprop, SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

checkpoint = ModelCheckpoint("fruit.h5", 
                            monitor="val_loss",
                            mode="min",
                            save_best_only=True,
                            verbose=1)
early_stop = EarlyStopping(monitor="val_loss", 
                          min_delta=0,
                          patience=0,
                          verbose=1,
                          restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                             factor=0.2,
                             patience=3,
                             verbose=1,
                             min_delta=0.0001)

In [13]:
callbacks = [early_stop, checkpoint]

model.compile(optimizer=RMSprop(lr=0.001), 
             loss="categorical_crossentropy",
             metrics=["accuracy"])

In [14]:
train_samples = 41322
test_samples = 13877
epochs = 5

In [15]:
history = model.fit_generator(train_generator, 
                    steps_per_epoch=train_samples // batch_size,
                       epochs=epochs,
                       callbacks=callbacks,
                       validation_data=validation_generator,
                       validation_steps=test_samples // batch_size)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
2582/2582 [==============================] - 1039s 402ms/step - loss: 1.5957 - accuracy: 0.5219 - val_loss: 2.2674e-04 - val_accuracy: 0.8574

Epoch 00001: val_loss improved from inf to 0.00023, saving model to fruit.h5
Epoch 2/5
2582/2582 [==============================] - 260s 101ms/step - loss: 0.5631 - accuracy: 0.8212 - val_loss: 5.5889e-04 - val_accuracy: 0.9061
Restoring model weights from the end of the best epoch

Epoch 00002: val_loss did not improve from 0.00023
Epoch 00002: early stopping


In [18]:
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model.predict_generator(validation_generator, test_samples // batch_size + 1)
y_pred = np.argmax(Y_pred, axis=1)
print(confusion_matrix(validation_generator.classes, y_pred))
# target_names = list(class_labels.values())

[[164   0   0 ...   0   0   0]
 [  0 163   0 ...   0   0   0]
 [  0  11 127 ...   0   0   0]
 ...
 [  0   0   0 ... 164   0   0]
 [  0   0   0 ...   0 127   0]
 [  0   0   0 ...   0   0 249]]


NameError: name 'class_labels' is not defined

In [19]:

print(classification_report(validation_generator.classes, y_pred))

              precision    recall  f1-score   support

           0       0.33      1.00      0.50       164
           1       0.78      0.99      0.87       164
           2       0.93      0.77      0.85       164
           3       0.82      0.14      0.24       161
           4       0.86      1.00      0.93       164
           5       0.68      1.00      0.81       164
           6       1.00      0.56      0.72       164
           7       0.76      0.31      0.44       144
           8       0.98      1.00      0.99       166
           9       1.00      0.91      0.95       164
          10       1.00      1.00      1.00       164
          11       1.00      1.00      1.00       143
          12       1.00      0.36      0.53       166
          13       0.98      0.98      0.98       166
          14       0.92      0.63      0.75       166
          15       0.83      0.93      0.88       166
          16       1.00      1.00      1.00       164
          17       0.64    